In [1]:
# 🖼️ ResNet18 + Gradio 圖片分類介面：AI 美食中文說明系統

import gradio as gr
import torch
import torch.nn as nn
from torchvision import transforms, models
from torchvision.models import ResNet18_Weights
from PIL import Image
import pandas as pd

# 裝置選擇
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 模型與說明表路徑
model_path = "resnet18_food.pth"
food_info_path = "C:/Users/echo6/Downloads/Food/food_info.csv"

# 圖片轉換
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# 載入食物說明表
food_df = pd.read_csv(food_info_path)
food_dict = dict(zip(food_df['food'], food_df['description']))
class_names = sorted(list(food_dict.keys()))

# 建立模型架構並載入訓練權重
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, len(class_names))
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval()
print("✅ ResNet18 模型已載入，準備啟動 Gradio")

# 預測函式

def predict_with_gradio(image):
    try:
        image = image.convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1)
            top3 = torch.topk(probs, 3)

            result_text = "📊 預測前 3 名：\n"
            for i in range(3):
                idx = top3.indices[0][i].item()
                label = class_names[idx]
                prob = top3.values[0][i].item()
                result_text += f"Top {i+1}: {label}（信心值：{prob:.2%}）\n"

            main_idx = top3.indices[0][0].item()
            main_label = class_names[main_idx]
            description = food_dict.get(main_label, "查無資料")

            result_text += f"\n🍽️ 預測食物（最可能）：{main_label}\n"
            result_text += f"📘 食物說明：\n{description}"
            return result_text
    except Exception as e:
        return f"❌ 發生錯誤：{str(e)}"


✅ ResNet18 模型已載入，準備啟動 Gradio


In [2]:
# Gradio 介面啟動
interface = gr.Interface(
    fn=predict_with_gradio,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="🍱 ResNet18 AI 美食分類與說明系統",
    description="上傳一張食物圖片，系統會分類並顯示 Top-3 結果與中文說明"
)

interface.launch()


* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.
